# Covariance Matrix preparation
In this notebook, we try to implement the covariance matrix preparation via ensemble average density matrix, as proposed in ["Covariance Matrix Preparation for Quantum Principal Component Analysis"](https://doi.org/10.1103/PRXQuantum.3.030334) by Gordon et al, 2022.

In [3]:
# library imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.linalg import expm
from collections import defaultdict
from IPython.display import display
from keras.datasets import mnist
import random

from qiskit_aer import *
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import UnitaryGate, PhaseEstimation
from qiskit.circuit.library.data_preparation import StatePreparation
from qiskit.quantum_info import DensityMatrix

import qPCA_funcs as QF

Loading the MNIST handwritten digit dataset:

In [4]:
#checking that the dataset loading works
(X, y), _ = mnist.load_data()

# take subset of 5000 images
idx = np.random.choice(np.arange(y.shape[0]), size=5000, replace=False)
X=X[idx,:,:]
y=y[idx]

#printing the shapes of the vectors 
print('X: ' + str(X.shape))
print('Y: ' + str(y.shape))

# check that the labels are balanced
nums, counts = np.unique(y, return_counts=True)
for n, c in zip(nums, counts):
    print(f"{n}: {c} counts")

# we need to flatten the images into a 1D vector each
# so X is a 2D array
X = X.reshape((X.shape[0], X.shape[1]*X.shape[2]))
print(X.shape)
# compute covariance matrix
cov_matr = np.cov(X, rowvar=False)
print(f'Covariance matrix shape: {cov_matr.shape}')

X: (5000, 28, 28)
Y: (5000,)
0: 505 counts
1: 545 counts
2: 498 counts
3: 484 counts
4: 481 counts
5: 463 counts
6: 478 counts
7: 556 counts
8: 507 counts
9: 483 counts
(5000, 784)
Covariance matrix shape: (784, 784)


Ok let's try something:

In [21]:
def amp_encode(vec):
    # vec is an input data vector
    # padded to len 2**n if necessary
    qc = QuantumCircuit(len(vec).bit_length()-1)
    qc.append(StatePreparation(vec), qc.qubits)
    return qc

# randomly sampling from ensemble
def estimate_rho_circuit(encoder_circuits, N_samples, shots=1):
    # given a list of state prep circuits for the data vectors
    # randomly picks one & runs it, returns the density matrix
    # which is averaged (no idea if this is even close to correct)
    rho_accum = None
    for _ in range(N_samples):
        i = random.randrange(len(encoder_circuits))
        qc = encoder_circuits[i]
        # directly simulates dm
        # or should I do this differently?
        dm = DensityMatrix.from_instruction(qc)
        rho_accum = dm if rho_accum is None else rho_accum + dm

    return rho_accum / N_samples

N_test = 10
Xtest = X[:N_test]
# normalize
norms = np.linalg.norm(Xtest, axis=1)
Xtest_normalized = Xtest/norms[:, np.newaxis]
# zero padding
dim = len(X[0])
new_dim =2**int(np.ceil(np.log2(dim)))
X_padded = np.zeros((N_test, new_dim))
X_padded[:, :dim] = Xtest_normalized
# create encoder circuits
encoder_circuits = [amp_encode(x) for x in X_padded]
# estimate rho
rho = estimate_rho_circuit(encoder_circuits, N_test)
# took 4890s to run

In [22]:
print(rho)

DensityMatrix([[ 3.03976788e-018-3.65495942e-019j,
                 3.63230388e-032+4.79363963e-032j,
                 5.94534574e-032+1.59081813e-032j, ...,
                -1.39327027e-121-8.60328948e-122j,
                -2.42406926e-122-2.26941629e-122j,
                -7.70316138e-136-1.10903065e-135j],
               [ 3.01812980e-032+4.56444694e-032j,
                -2.15004966e-033+1.76673922e-034j,
                 5.94670553e-046+4.30420134e-046j, ...,
                -1.35559720e-123-1.38440462e-123j,
                -1.72480329e-135-5.00488442e-136j,
                -1.49714449e-137-1.29940698e-137j],
               [ 5.37165201e-032+2.79918091e-032j,
                 6.67669454e-046+4.81970038e-046j,
                -1.30865835e-033-7.60266093e-034j, ...,
                -1.49730706e-135-7.29682582e-136j,
                -1.22696938e-123-5.19678424e-124j,
                -3.84662371e-139-6.64717112e-138j],
               ...,
               [ 1.44655175e-122-1.18142347e

In [26]:
# let's test this:
U_mat = expm(-1j * rho * 2*np.pi)
U = UnitaryGate(U_mat, label="U")

In [ ]:
nsys = int(np.log2(new_dim))
resolution = 4
verbose = True
qc_init = QuantumCircuit(nsys)
simulator = AerSimulator(method="density_matrix")
pe_circuit = QF.build_PE_circuit(qc_init, U, resolution=resolution, verbose=verbose)
# transpile the circuit
pe_transpiled = transpile(pe_circuit, simulator)
# run circuit
pe_result = simulator.run(pe_transpiled, initial_state=rho, shots=n_shots).result()
# get counts
pe_counts = pe_result.get_counts()

# plotting the counts if desired
if plot_results:
    print("PE result counts:")
    plot = plot_histogram(pe_counts, figsize=(25, 5))
    display(plot)

# compute probailities from counts
if verbose: print("\nProbability estimation...")
probabilities = QF.probability_estimation(pe_counts, pe_circuit.num_qubits, n_shots)

In [ ]:
# rest to follow